### Data clean up in the database
- removed 14 topics in {_, aa, aaaaa, ...}
- delete from sentiments where length(sentiments.topic) < 2 (282 removed)

### Topics per Taster
| Topic | Label | TB Pol. | SIA Comp. | N Occurrences |
| --- | --- | --- | --- | --- |
| people | 0 | -0.1 | 0.2732 | 1958 |
| film | 0 | 0.59 | 0.9664 | 1201 |
| movie | 0 | 0.26125 | 0.3612 | 1032 |
| guys | 0 | 0.13342151675485 | 0.9956 | 717 |
| friends | 0 | 0.00426136363636 | 0.7713 | 580 |
| things | 0 | 0.15555555555556 | 0.8694 | 464 |
| life | 0 | 0.18 | 0.5574 | 439 |
| friend | 0 | 0.13342151675485 | 0.9956 | 403 |
| parents | 0 | 0.15 | 0.2477 | 397 |
| girl | 0 | -0.2 | -0.6486 | 385 |
| man | 0 | 0.24444444444444 | -0.4416 | 378 |
| thing | 0 | -0.0875 | -0.296 | 356 |
|  |  |  |  |  |
| people | 1 | 0.14675324675325 | 0.3884 | 5520 |
| life | 1 | 0.14675324675325 | 0.3884 | 4106 |
| things | 1 | -0.14768170426065 | -0.9464 | 2413 |
| friends | 1 | -0.12142857142857 | -0.8463 | 2122 |
| parents | 1 | 0.08605442176871 | -0.2975 | 1480 |
| family | 1 | 0.00923558501684 | -0.4778 | 1298 |
| person | 1 | -0.09309523809524 | -0.9921 | 1207 |
| thing | 1 | -0.30416666666667 | -0.98 | 1128 |
| friend | 1 | -0.12142857142857 | -0.8463 | 1106 |
| time | 1 | -0.28 | -0.7519 | 1056 |
| mom | 1 | -0.2 | -0.5588 | 1019 |
| world | 1 | -0.30416666666667 | -0.98 | 763 |
| N (sent > +/- 0.3, topics/doc > 4) |  |  |  | 100110 |

In [ ]:
import pandas as pd
import sqlite3

connection = None
try:
    connection = sqlite3.connect("data/mental_health.db")
except sqlite3.Error as e:
    print(e)

topics_df = pd.read_sql_query('''
    select id, text, label, topic, (sia_compound + tb_polarity)/2 as sentiment
    from texts, sentiments
    where id = document
    and id in (
        select document
        from (
            select *
            from texts, (
                select document, topic, (sia_compound + tb_polarity)/2 as sentiment
                from sentiments
            ) as sentiments_computed
            where id = sentiments_computed.document
            and (sentiments_computed.sentiment > 0.3 or sentiments_computed.sentiment < -0.3) -- some level of polarity
        ) as sentiments_filtered
        group by sentiments_filtered.document
        having count(sentiments_filtered.topic) > 4 -- texts containing at least N topics
    )
''', connection)
topics_df.head()

In [ ]:
pairs_df = pd.DataFrame(columns=["document", "label", "text_lhs", "topic_lhs", "sentiment_lhs", "text_rhs", "topic_rhs", "sentiment_rhs", "distance"])

topics_df = topics_df.sample(1000, random_state=17)
#pairs_df.head()

In [ ]:
import math
def compute_distance(lhs, rhs):
    return math.sqrt((lhs - rhs) ** 2)
    # return 1

In [ ]:
from tqdm import tqdm

i = 0

for index_lhs, row_lhs in tqdm(topics_df.iterrows()):
    for index_rhs, row_rhs in topics_df.iterrows():
        if  ((index_lhs < index_rhs) & 
             (row_lhs["id"] == row_rhs["id"]) & 
             (row_lhs["topic"] != row_rhs["topic"])):
            
            new_row = pd.DataFrame({
                "document": row_lhs["id"],
                "label": row_lhs["label"],
                "text_lhs": row_lhs["text"],
                "topic_lhs": row_lhs["topic"],
                "sentiment_lhs": row_lhs["sentiment"],
                "text_rhs": row_rhs["text"],
                "topic_rhs": row_rhs["topic"],
                "sentiment_rhs": row_rhs["sentiment"],
                "distance": compute_distance(row_lhs["sentiment"], row_rhs["sentiment"])
            }, index=[i])
            pairs_df = pd.concat([new_row, pairs_df])
            i = i + 1

pairs_df.head()

In [ ]:
import sqlite3

connection = None
try:
    connection = sqlite3.connect("data/mental_health.db")
except sqlite3.Error as e:
    print(e)

pairs_df.to_sql("pairs", connection, if_exists="append", index=False)

connection.commit()